In [1]:
include("functions.jl")

magic = readdlm("magic04.data", ',', Any, '\n')
magic = magic[sample(1:19020, 200, replace = false),:]
loc = findall(x->x=="g",magic[:,11])
magic[loc,11] .= 1
loc = findall(x->x=="h",magic[:,11])
magic[loc,11] .= 0
magic = convert(Array{Float64}, magic)

class = magic[:,11]
magic[:,11] .= 1
n, p = size(magic)

(200, 11)

Magic gamma telescope dataset
- \# of sample: 19,020
- \# of covariate: 11 (including intercept)
- response: binary

Choose $N_1$ random samples, and obtain $\hat\beta$ for the logistic regression model. Then, find a locally {$A_K$, $D$}-optimal design of sample size $N_2$ with $\hat\beta$.

#### Case 2, TED
- $D_K$-optimality
- $N_1$: 30
- $N_2$: 100

In [ ]:
using BenchmarkTools, StatsBase

Random.seed!(1)
N1 = 30
N2 = 100
samp1 = sample(1:n, N1, replace = false)
cand = setdiff(1:n, samp1)

@time aopt = sagnol_D(magic[cand,:], magic[samp1,:], N2; K = magic', verbose=1, IC=1)
aopt = BitArray(round.(aopt))
println(sum(aopt))
samp2 = [samp1; cand[aopt]]
exact = det((magic[samp2,:]'magic[samp2,:])\(magic')))

Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 199911          
  Cones                  : 34402           
  Scalar variables       : 257982          
  Matrix variables       : 0               
  Integer variables      : 171             

Optimizer started.
Mixed integer optimizer started.
Threads used: 24
Presolve started.
Presolve terminated. Time = 4.75
Presolved problem: 163276 variables, 71005 constraints, 900377 non-zeros
Presolved problem: 0 general integer, 170 binary, 163106 continuous
Clique table size: 0
BRANCHES RELAXS   ACT_NDS  DEPTH    BEST_INT_OBJ         BEST_RELAX_OBJ       REL_GAP(%)  TIME  
0        2        0        0        NA                   NA                   NA          27.6  
Cut generation started.
Cut generation terminated. Time = 13.39
7        17       8        2        NA                   1.0000000000e+300    NA 

In [ ]:
@time aopt = sagnol_D(magic[cand,:], magic[samp1,:], N2; K = magic', verbose=1, IC=0)

n_try = 30

apprx = zeros(Float64, n_try)
apprx_size = zeros(Int64, n_try)
for i = 1:n_try
    bool_opt = rand(n-N1) .< aopt
    apprx_size = sum(bool_opt)
    samp2 = [samp1; cand[bool_opt]]
    apprx[i] = det(magic*((magic[samp2,:]'magic[samp2,:])\(magic')))
end

countmap(apprx_size)

In [ ]:
describe(apprx)

In [ ]:
#N1 = 30
#N2 = 100
#samp1 = sample(1:n, N1, replace = false)
#cand = setdiff(1:n, samp1)

@time aopt = sagnol_D(magic[cand,:], magic[samp1,:], N2; K = magic', verbose=0, IC=1)
aopt = BitArray(round.(aopt))
println(sum(aopt))
samp2 = [samp1; cand[aopt]]
exact = det((magic[samp2,:]'magic[samp2,:])\(magic')))

In [ ]:
@time aopt = sagnol_D(magic[setdiff(1:n, samp1),:], magic[samp1,:], N2; K = magic', verbose=0, IC=0)

n_try = 30

apprx = zeros(Float64, n_try)
apprx_size = zeros(Int64, n_try)
for i = 1:n_try
    bool_opt = rand(n-N1) .< aopt
    apprx_size = sum(bool_opt)
    samp2 = [samp1; cand[bool_opt]]
    apprx[i] = det(magic*((magic[samp2,:]'magic[samp2,:])\(magic')))
end

countmap(apprx_size)

In [ ]:
describe(apprx)